# Building a research Agent with LangChain

In [1]:
from langchain_openai.chat_models import ChatOpenAI

In [2]:
llm_chat = ChatOpenAI(temperature=0)

In [6]:
from langchain.utilities import SerpAPIWrapper
from langchain.tools import tool

search = SerpAPIWrapper()

@tool
def web_search(query: str):
    """Searches the web given a user query and returns the most relevant links"""
    output = search.results(query)
    results = output["organic_results"]
    return [r["link"] for r in results]

web_search("Artificial Intelligence")

['https://en.wikipedia.org/wiki/Artificial_intelligence',
 'https://www.ibm.com/topics/artificial-intelligence',
 'https://www.techtarget.com/searchenterpriseai/definition/AI-Artificial-Intelligence',
 'https://builtin.com/artificial-intelligence',
 'https://cloud.google.com/learn/what-is-artificial-intelligence',
 'https://www.investopedia.com/terms/a/artificial-intelligence-ai.asp']

In [ ]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")

prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(

In [ ]:
prompt.input_variables

['agent_scratchpad', 'input']

In [ ]:
prompt.input_types

{'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]],
 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}

In [ ]:
prompt.__dict__

{'name': None,
 'input_variables': ['agent_scratchpad', 'input'],
 'input_types': {'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]],
  'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]},
 'output_parser': None,
 'partial_variables': {},
 'messages': [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
  MessagesPlaceholder(variable_name='chat_history', optional=True),
  HumanMessagePromptTemplate(prompt=PromptTemplate(input

In [36]:
from langchain.prompts import ChatPromptTemplate


template = """
{agent_scratchpad}.
You are a helpful research assistant.
Given a query you will use the available tools or your own knowledge to 
answer the question by providing a comprehensive report on the matter.
Here is the user query:\n\n'''{query}'''.
"""
prompt = ChatPromptTemplate.from_template(template)

In [40]:
prompt.format(query="What is the best way to learn a new language?", agent_scratchpad=[])

"Human: \n[].\nYou are a helpful research assistant.\nGiven a query you will use the available tools or your own knowledge to \nanswer the question by providing a comprehensive report on the matter.\nHere is the user query:\n\n'''What is the best way to learn a new language?'''.\n"

To build an agent I need to do 2 essential things:

1. Connect a model to tools
2. Set up a loop in which the model continuously tries to solve the problem by either just calling the llm or calling a tool, observing the result and repeating until finished.

In [41]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'query'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'query'], template="\n{agent_scratchpad}.\nYou are a helpful research assistant.\nGiven a query you will use the available tools or your own knowledge to \nanswer the question by providing a comprehensive report on the matter.\nHere is the user query:\n\n'''{query}'''.\n"))])

In [42]:
from langchain.agents import create_openai_tools_agent

tools = [web_search]

react_agent = create_openai_tools_agent(llm_chat, tools, prompt)

In [43]:
from langchain.agents import AgentExecutor


react_agent_executor = AgentExecutor(agent=react_agent, tools=tools, verbose=True)

In [44]:
react_agent_executor.invoke({"query": "What is the best way to learn a new language?",
                             "agent_scratchpad": []})



> Entering new AgentExecutor chain...
To learn a new language effectively, there are several strategies and techniques that can be helpful. Here are some of the best ways to learn a new language:

1. Immerse Yourself in the Language: One of the most effective ways to learn a new language is to immerse yourself in it as much as possible. This can be done by living in a country where the language is spoken, or by surrounding yourself with native speakers of the language. Immersion helps you to pick up the language naturally and develop your listening and speaking skills.

2. Take Language Classes: Taking language classes can provide you with a structured learning environment and guidance from a qualified teacher. Language classes can help you learn the grammar, vocabulary, and pronunciation of the language. Look for classes that focus on conversation practice to improve your speaking skills.

3. Practice Regularly: Consistency is key when learning a new language. Set aside regular time

{'query': 'What is the best way to learn a new language?',
 'agent_scratchpad': [],
 'output': "To learn a new language effectively, there are several strategies and techniques that can be helpful. Here are some of the best ways to learn a new language:\n\n1. Immerse Yourself in the Language: One of the most effective ways to learn a new language is to immerse yourself in it as much as possible. This can be done by living in a country where the language is spoken, or by surrounding yourself with native speakers of the language. Immersion helps you to pick up the language naturally and develop your listening and speaking skills.\n\n2. Take Language Classes: Taking language classes can provide you with a structured learning environment and guidance from a qualified teacher. Language classes can help you learn the grammar, vocabulary, and pronunciation of the language. Look for classes that focus on conversation practice to improve your speaking skills.\n\n3. Practice Regularly: Consisten

In [45]:
react_agent_executor.invoke({"query": "What is the best way to learn good programming practices?",
                             "agent_scratchpad": []})



> Entering new AgentExecutor chain...
To learn good programming practices, there are several approaches you can take. Here are some of the best ways to learn and improve your programming skills:

1. Read Books and Online Resources: There are many books and online resources available that cover programming best practices. Some popular books include "Clean Code" by Robert C. Martin and "The Pragmatic Programmer" by Andrew Hunt and David Thomas. These resources provide valuable insights and guidelines on writing clean, maintainable, and efficient code.

2. Take Online Courses: Online learning platforms like Coursera, Udemy, and edX offer a wide range of programming courses that cover best practices. Look for courses that focus on software engineering principles, code quality, and design patterns. These courses often include practical exercises and assignments to help you apply what you've learned.

3. Practice Regularly: The more you practice programming, the better you will become. Set

{'query': 'What is the best way to learn good programming practices?',
 'agent_scratchpad': [],
 'output': 'To learn good programming practices, there are several approaches you can take. Here are some of the best ways to learn and improve your programming skills:\n\n1. Read Books and Online Resources: There are many books and online resources available that cover programming best practices. Some popular books include "Clean Code" by Robert C. Martin and "The Pragmatic Programmer" by Andrew Hunt and David Thomas. These resources provide valuable insights and guidelines on writing clean, maintainable, and efficient code.\n\n2. Take Online Courses: Online learning platforms like Coursera, Udemy, and edX offer a wide range of programming courses that cover best practices. Look for courses that focus on software engineering principles, code quality, and design patterns. These courses often include practical exercises and assignments to help you apply what you\'ve learned.\n\n3. Practice Re

In [47]:
react_agent_executor.invoke({"query": "Research the current most popular open source frameworks to build llm-based agents in Python and return 5 releavnt urls.",
                             "agent_scratchpad": []})



> Entering new AgentExecutor chain...

Invoking: `web_search` with `{'query': 'current most popular open source frameworks to build llm-based agents in Python'}`


['https://github.com/Hannibal046/Awesome-LLM', 'https://www.simform.com/blog/open-source-llm/', 'https://odsc.medium.com/9-open-source-llms-and-agents-to-watch-728049d77060', 'https://www.unite.ai/best-open-source-llms/', 'https://neptune.ai/blog/best-open-source-mlops-tools', 'https://www.reddit.com/r/LocalLLaMA/comments/17m2lql/best_framework_for_llm_based_applications_in/', 'https://www.taskade.com/blog/top-autonomous-agents/', 'https://www.marktechpost.com/2024/01/03/10-open-source-tools-for-llm-applications-development/', 'https://www.datacamp.com/blog/top-open-source-llms', 'https://eng-shamsan.medium.com/large-language-model-evaluation-tools-2023-open-source-and-commercial-567525c57f82']I have found some relevant URLs for you to explore:

1. [Awesome-LLM](https://github.com/Hannibal046/Awesome-LLM)
2. [Open Source L

{'query': 'Research the current most popular open source frameworks to build llm-based agents in Python and return 5 releavnt urls.',
 'agent_scratchpad': [],
 'output': 'I have found some relevant URLs for you to explore:\n\n1. [Awesome-LLM](https://github.com/Hannibal046/Awesome-LLM)\n2. [Open Source LLM](https://www.simform.com/blog/open-source-llm/)\n3. [9 Open Source LLMs and Agents to Watch](https://odsc.medium.com/9-open-source-llms-and-agents-to-watch-728049d77060)\n4. [Best Open Source LLMs](https://www.unite.ai/best-open-source-llms/)\n5. [Best Open Source MLOps Tools](https://neptune.ai/blog/best-open-source-mlops-tools)\n\nYou can visit these URLs to learn more about the current most popular open source frameworks to build LLM-based agents in Python.'}

Now let's build this same agent but using a more in depth approach going one level down in the abstraction layers.